In [1]:
import pandas as pd
import numpy as np

In [2]:
!pwd

/Users/emilian/Proyecto_Cruz_Roja/code


In [3]:
first = pd.read_csv('~/Proyecto_Cruz_Roja/data/RawData/diabetes.csv')

In [4]:
first

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [5]:
first['genero'] = 'Femenino'

In [6]:
columns = ['Age', 'genero', 'BloodPressure', 'Glucose', 'BMI', 'Outcome']
first = first[columns]

In [7]:
first=first.rename(columns={"Age": "edad", "Glucose": "glucosa", 'BMI':'IMC', 'Outcome':'Class'})

In [8]:

first['Class'] = first['Class'].map({1: 'Y', 0:'N'})

#one=first[columns]
first.to_csv('~/Proyecto_Cruz_Roja/data/TrainingData/Diabetes1.csv')

In [9]:
import pandas as pd
import numpy as np
#plotting
import seaborn as sns
import matplotlib.pyplot as plt
#sklearn
from sklearn.datasets import load_diabetes #importing data
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures
#scipy
from scipy.stats import loguniform
from pandasql import sqldf

In [10]:
#importing dataset
diab = load_diabetes()
#defininf feature and label
X = diab['data']
y = diab['target']
#printing shapes
print(X.shape, y.shape)

(442, 10) (442,)


In [11]:
df = pd.DataFrame(data=diab.data, columns=diab.feature_names)
df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641


In [12]:
#creating dataframe from X
df = pd.DataFrame(X, columns=["edad","genero","IMC","BloodPressure", "tc", "ldl", "hdl","tch", "ltg", "glucosa"])
#adding 'progression' from y
df['progression'] = diab['target']
#showing head
df.head()

,edad,genero,IMC,BloodPressure,tc,ldl,hdl,tch,ltg,glucosa,progression
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641,135.0


In [13]:

columns = ['edad', 'genero', 'BloodPressure', 'glucosa', 'IMC']
df[columns]

,edad,genero,BloodPressure,glucosa,IMC
0,0.038076,0.050680,0.021872,-0.017646,0.061696
1,-0.001882,-0.044642,-0.026328,-0.092204,-0.051474
2,0.085299,0.050680,-0.005671,-0.025930,0.044451
3,-0.089063,-0.044642,-0.036656,-0.009362,-0.011595
4,0.005383,-0.044642,0.021872,-0.046641,-0.036385
...,...,...,...,...,...
437,0.041708,0.050680,0.059744,0.007207,0.019662
438,-0.005515,0.050680,-0.067642,0.044485,-0.015906
439,0.041708,0.050680,0.017282,0.015491,-0.015906
440,-0.045472,-0.044642,0.001215,-0.025930,0.039062


In [14]:
data = pd.read_csv("~/Proyecto_Cruz_Roja/data/RawData/DatasetofDiabetes.csv")
data =data.rename(columns={"AGE": "edad", "Gender":"genero", "HbA1c": "glucosa", 'BMI':'IMC'})

In [15]:

data

,ID,No_Pation,genero,edad,Urea,Cr,glucosa,Chol,TG,HDL,LDL,VLDL,IMC,CLASS
0,502,17975,F,50,4.7,46,4.9,4.2,0.9,2.4,1.4,0.5,24.0,N
1,735,34221,M,26,4.5,62,4.9,3.7,1.4,1.1,2.1,0.6,23.0,N
2,420,47975,F,50,4.7,46,4.9,4.2,0.9,2.4,1.4,0.5,24.0,N
3,680,87656,F,50,4.7,46,4.9,4.2,0.9,2.4,1.4,0.5,24.0,N
4,504,34223,M,33,7.1,46,4.9,4.9,1.0,0.8,2.0,0.4,21.0,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,200,454317,M,71,11.0,97,7.0,7.5,1.7,1.2,1.8,0.6,30.0,Y
996,671,876534,M,31,3.0,60,12.3,4.1,2.2,0.7,2.4,15.4,37.2,Y
997,669,87654,M,30,7.1,81,6.7,4.1,1.1,1.2,2.4,8.1,27.4,Y
998,99,24004,M,38,5.8,59,6.7,5.3,2.0,1.6,2.9,14.0,40.5,Y


In [16]:
columns = ['edad', 'genero', 'glucosa', 'IMC', 'CLASS']
data=data.replace('F', 'Femenino').replace('M', 'Masculino')
#data=data.replace('M', 'Masculino')
second=data[columns]
second.to_csv('~/Proyecto_Cruz_Roja/data/TrainingData/Diabetes2.csv')
